<a href="https://colab.research.google.com/github/KKeun-B/learning/blob/main/2020/CUDA_nvprof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA C/C++ 통합 메모리(Unified Memory)와 nvprof을 이용한 가속 애플리케이션 메모리 관리



본 강좌와 다른 CUDA 기초 강좌의 후속 학습 자료로서 강력히 추천하는 [CUDA 베스트 프렉티스 가이드](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations)에서는 **APOD**(**A**sess 평가, **P**arallelize 병렬화, **O**ptimize 최적화, **D**eploy 배포)라고 불리우는 디자인 사이클을 추천하고 있습니다. 간단히 말해서 APOD는 반복적 디자인 프로세스를 규정하는데, 개발자들은 애플리케이션 성능을 가속화하고 코드를 배포하는 데에 있어 점진적 개선 방식을 적용할 수 있습니다. 개발자들이 점점 유능한 CUDA 프로그래머가 되어감에 따라 더 진보한 최적화 기술을 가속화 코드베이스에 적용할 수 있게 됩니다.

본 강좌에서는 이러한 반복적 개발 방법론을 이용할 것입니다. 여러분은 **NVIDIA 커맨드라인 프로파일러**를 사용하여 애플리케이션의 성능을 정량적으로(역주: 원문에는 qualitatively이나 quantitative의 오기인 듯) 측정하고, 최적화 기회를 포착하며, 새로운 기법을 배우고 사이클을 반복하기 전에 점진적 개선을 적용하게 될 것입니다. 본 강좌의 핵심은 여러분이 배우고 적용할 다수의 기법들이 CUDA의 **통합 메모리(Unified Memory; UM)**의 동작에 대한 것이라는 점입니다. CUDA 개발자들에게 통합 메모리의 동작 이해는 필수적이며 다른 많은 메모리 관리 기법을 이해하기 위한 발판이 됩니다.




---

## 선행지식

강좌의 이해를 위하여 여러분은 다음 사항을 알고 있어야 합니다.

* CPU 함수 호출과 GPU 커널 구동을 포함한 C/C++ 프로그램의 작성, 컴파일, 실행하기
* 실행 설정을 이용한 병렬 스레드 계층구조 제어하기
* 직렬 반복문을 GPU에서 병렬로 실행하도록 코드 수정하기
* 통합 메모리의 할당과 해제



---

## 학습목표

강좌를 마치면 여러분은 다음 사항을 수행할 수 있게 될 것입니다.

* **NVIDIA 커맨드라인 프로파일러(nvprof)**를 이용하여 가속화 애플리케이션의 성능 프로파일링하기
* 실행 설정 최적화를 위한 **스트리밍 멀티프로세서**에 대한 체계적 이해
* 페이지 폴트와 데이터 마이그레이션에 관련된 **통합 메모리**의 동작 이해
* 페이지 폴트와 데이터 마이그레이션을 감소시켜 성능을 향상시키는 **비동기적 메모리 프리패칭** 사용
* 반복적 개발 사이클을 적용하여 애플리케이션 가속화와 배치를 신속히 진행하기



---

## NVIDIA 프로파일러를 활용한 최적화


### 실습: nvprof을 이용하여 애플리케이션 프로파일링 하기

가속화 코드 베이스의 최적화 시도가 실제로 성공했는가를 확인하는 방법 중의 하나로서 성능과 관련된 정량적 정보를 프로파일링하는 방법이 있습니다. **`nvprof`**는 이러한 작업을 지원하는 NVIDIA의 커맨드라인 프로파일러입니다. CUDA 툴킷과 함께 배포되는 이것은 가속화 애플리케이션을 프로파일링하는 강력한 도구입니다.

`nvprof`를 사용하기는 쉽습니다. 가장 기본적인 사용법은 `nvcc`로 컴파일한 실행 파일의 경로를 전달해 주는 것입니다. `nvprof`는 해당 애플리케이션을 실행한 후 GPU 활동, CUDA API 호출 기록, **통합 메모리(Unified Memory)** 활동 정보 등을 정리하여 출력해 줍니다. 자세한 내용은 강좌를 진행하며 다루도록 하겠습니다.

애플리케이션을 가속화하거나 이미 가속화된 애플리케이션을 최적화할 때에 과학적이고 반복적인 접근법을 취하세요. 애플리케이션 변경 후에는 프로파일링을 수행하고, 기록을 남기고, 수정사항이 성능에 미치는 의미를 적으세요. 보다 이른 단계에, 그리고 자주 이러한 관찰을 수행하세요. 작은 노력들이 모여 성능을 획기적으로 향상시키고 출시를 돕는 경우가 많이 있습니다. 빈번한 프로파일링은 특정한 코드 변경이 실제 성능에 어떠한 영향을 미치는가를 여러분에게 가르쳐 줄 것입니다. 이러한 지식은 코드 베이스를 한참 변경한 후에 수행하는 프로파일링으로는 얻을 수 없는 중요한 자산입니다.  




### 실습: nvprof을 이용하여 애플리케이션 프로파일링 하기

[`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)(<---- 소스 파일을 새로운 탭에서 열고 수정하려면 클릭하세요.)는 고속화 벡터합 프로그램입니다. 아래 두 개의 코드 실행 셀을 사용하세요. (`CTRL+ENTER`를 누르면 됩니다.) 첫 번째 코드 실행 셀은 벡터합 프로그램을 컴파일하고 실행합니다. 두 번째 셀은 방금 컴파일한 실행 파일을 `nvprof`을 이용하여 프로파일링합니다.

애플리케이션의 프로파일링을 수행한 후, 출력에 나온 정보를 이용하여 아래 질문에 답하세요.

* 여기서 호출된 유일한 CUDA 커널의 이름은 무엇인가요?
* 커널은 몇 번 실행됐나요?
* 커널이 실행되는 데 얼마나 걸렸나요? 시간을 기록해 두세요. 나중에 최적화를 수행하면 얼마나 빨라졌는지 알고 싶으실 것입니다.

In [1]:
%%writefile 01-vector-add.cu
#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = 1;
  numberOfBlocks = 1;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}

Writing 01-vector-add.cu


In [2]:
!nvcc -o single-thread-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [3]:
!nvprof ./single-thread-vector-add

==140== NVPROF is profiling process 140, command: ./single-thread-vector-add
Success! All values calculated correctly.
==140== Profiling application: ./single-thread-vector-add
==140== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.30219s         1  2.30219s  2.30219s  2.30219s  addVectorsInto(float*, float*, float*, int)
      API calls:   86.65%  2.30221s         1  2.30221s  2.30221s  2.30221s  cudaDeviceSynchronize
                   12.50%  332.14ms         3  110.71ms  24.583us  332.06ms  cudaMallocManaged
                    0.82%  21.706ms         3  7.2354ms  6.3716ms  8.6701ms  cudaFree
                    0.02%  435.89us         1  435.89us  435.89us  435.89us  cuDeviceTotalMem
                    0.01%  172.99us       101  1.7120us     160ns  71.575us  cuDeviceGetAttribute
                    0.00%  69.024us         1  69.024us  69.024us  69.024us  cudaLaunchKernel
                    0.00%  3

### 실습: 최적화와 프로파일링 하기

[`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)가 단일 스레드 블록상의 다수의 스레드에서 실행되도록 실행 설정을 수정하세요. 아래 코드 실행 셀에서 재컴파일하고 `nvprof`을 이용하여 실행 파일을 프로파일링하세요. 프로파일링 출력을 이용하여 커널의 실행 시간을 찾아 보세요. 최적화를 통한 속도 향상이 어느 정도인가요? 결과를  기록하세요.

In [4]:
%%writefile 01-vector-add.cu



#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = 256;
  numberOfBlocks = 128;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}




Overwriting 01-vector-add.cu


In [5]:
!nvcc -o multi-thread-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [6]:
!nvprof ./multi-thread-vector-add

==190== NVPROF is profiling process 190, command: ./multi-thread-vector-add
Success! All values calculated correctly.
==190== Profiling application: ./multi-thread-vector-add
==190== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  116.31ms         1  116.31ms  116.31ms  116.31ms  addVectorsInto(float*, float*, float*, int)
      API calls:   68.73%  306.29ms         3  102.10ms  14.905us  306.22ms  cudaMallocManaged
                   26.10%  116.32ms         1  116.32ms  116.32ms  116.32ms  cudaDeviceSynchronize
                    5.03%  22.421ms         3  7.4737ms  6.7685ms  8.8603ms  cudaFree
                    0.08%  377.78us         1  377.78us  377.78us  377.78us  cuDeviceTotalMem
                    0.04%  158.58us       101  1.5700us     134ns  67.454us  cuDeviceGetAttribute
                    0.01%  60.642us         1  60.642us  60.642us  60.642us  cudaLaunchKernel
                    0.01%  32.

### 실습: 반복적 최적화 하기

본 실습에서 여러분은 몇 차례에 걸쳐 [`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)의 실행 설정을 수정, 프로파일링, 기록함으로써 성능에 미치는 영향을 살펴 볼 것입니다. 아래 가이드라인을 따르세요.

* 실행 설정을 업데이트할 3~5 가지 방법을 나열하되, 다양한 범위의 그리드 및 블록 크기 조합을 사용하세요.
* [`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)를 수정하여 나열된 방법들을 하나씩 시도하세요.
* 아래 셀을 이용하여 컴파일과 프로파일링을 수행하세요.
* 프로파일링 출력에 주어진 실행 시간을 기록하세요.
* 나열된 방법들 각각에 대한 수정/프로파일/기록 사이클을 수행하세요.

어느 실행 설정이 가장 빠른 결과를 보였나요?

In [7]:
%%writefile 01-vector-add.cu

#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = 1024;
  numberOfBlocks = 256;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}


Overwriting 01-vector-add.cu


In [8]:
!nvcc -o iteratively-optimized-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [9]:
!nvprof ./iteratively-optimized-vector-add

==240== NVPROF is profiling process 240, command: ./iteratively-optimized-vector-add
Success! All values calculated correctly.
==240== Profiling application: ./iteratively-optimized-vector-add
==240== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  124.99ms         1  124.99ms  124.99ms  124.99ms  addVectorsInto(float*, float*, float*, int)
      API calls:   68.18%  316.25ms         3  105.42ms  14.910us  316.17ms  cudaMallocManaged
                   26.95%  125.00ms         1  125.00ms  125.00ms  125.00ms  cudaDeviceSynchronize
                    4.73%  21.942ms         3  7.3139ms  6.6601ms  8.5521ms  cudaFree
                    0.09%  400.60us         1  400.60us  400.60us  400.60us  cuDeviceTotalMem
                    0.04%  165.68us       101  1.6400us     152ns  71.189us  cuDeviceGetAttribute
                    0.01%  58.775us         1  58.775us  58.775us  58.775us  cudaLaunchKernel
            

---

## 스트리밍 멀티프로세서와 디바이스 질의하기

본 섹션에서는 GPU 하드웨어의 특정 기능을 이해하고 이것을 최적화에 활용하는 법을 살펴 봅니다. 먼저 **스트리밍 멀티프로세서**를 살펴 보고 앞에서 작업한 가속 벡터합 프로그램을 좀 더 최적화해 봅니다.

다음 슬라이드는 앞으로 공부할 내용을 개략적인 수준에서 시각적으로 보여줍니다. 주제를 보다 상세히 다루기에 앞서 슬라이드를 클릭하시면서 살펴 보세요.

In [10]:
%%HTML

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vRByDOlhmGKNY9IgFonAhE-uM0NAPdZGo8v8vlBBPqRB7RDx-E5g0OnGOpC2VoO-eWFhZBWv5yCtGfk/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

---
### 스트리밍 멀티프로세서와 워프

CUDA 애플리케이션을 실행하는 GPU는 **스트리밍 멀티프로세서(streaming multiprocessor; SM)**라는 프로세싱 유닛을 가지고 있습니다. 최대한 많은 수의 병렬처리를 위하여 GPU에 있는 *SM 숫자의 배수로 이루어진 블록수를 그리드의 크기로 선택*함으로써 성능 이득을 얻을 수 있습니다.

SM은 하나의 블록 안에 있는 32 개의 스레드를 하나의 그룹처럼 다루어 스레드 생성, 관리, 스케쥴링, 실행을 합니다. 이러한 32 개의 스레드로 이루어진 그룹을 **워프([warp](https://en.wikipedia.org/wiki/Thread_block_(CUDA_programming)#Warps))**라고 합니다. [SM과 warp에 대한 상세 내용](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation)은 본 강좌의 범위를 벗어나지만 *32의 배수를 스레드 개수로 갖는 블록 크기를 선택*함으로써 성능 이득을 얻을 수 있다는 것을 기억하는 것은 중요합니다. 

### 프로그램으로 GPU 디바이스 속성 질의하기

GPU 상의 SM 개수는 GPU 모델에 따라 다르기 때문에, 서로 다른 SM 개수를 가진 GPU 간의 프로그램 이식성을 유지하기 위하여 SM 개수는 코드 베이스에 하드코드되어서는 안됩니다. 이 정보는 프로그램으로 얻어내야 합니다.

아래 예제는 CUDA C/C++에서, SM 값을 포함하여, 현재 활성화된 GPU의 다양한 속성을 알아내는 데 사용되는 C 구조체를 얻는 방법을 보여줍니다. 

```cpp
int deviceId;
cudaGetDevice(&deviceId);                  // `deviceId` now points to the id of the currently active GPU.

cudaDeviceProp props;
cudaGetDeviceProperties(&props, deviceId); // `props` now has many useful properties about
                                           // the active GPU device.
```



### 실습: 디바이스 질의하기

[`01-get-device-properties.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/04-device-properties/01-get-device-properties.cu)는 다수의 초기화되지 않은 변수를 포함하고 있기 때문에, 현재 활성화된 GPU의 상세한 속성을 표시해야 하는 부분에서 의미 없는 값을 출력하고 있습니다. 

[`01-get-device-properties.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/04-device-properties/01-get-device-properties.cu) 코드를 수정하여 소스 코드가 의도했던 바대로 디바이스 속성의 실제값을 출력하도록 만드세요. [CUDA 런타임 문서](http://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html)를 읽어 보시면 실습을 진행하는 데 필요한 도움을 얻을 수 있습니다. 어떻게 해야 할 지 모르는 경우에는 ***(솔루션      : 01-get-device-properties-solution.cu)***을 참고하세요. 

In [11]:
%%writefile 01-get-device-properties.cu
#include <stdio.h>

int main()
{
  /*
   * Assign values to these variables so that the output string below prints the
   * requested properties of the currently active GPU.
   */

  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId); 
  int computeCapabilityMajor = props.major;
  int computeCapabilityMinor = props.minor;
  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;

  /*
   * There should be no need to modify the output string below.
   */

  printf("Device ID: %d\nNumber of SMs: %d\nCompute Capability Major: %d\nCompute Capability Minor: %d\nWarp Size: %d\n", deviceId, multiProcessorCount, computeCapabilityMajor, computeCapabilityMinor, warpSize);
}


Writing 01-get-device-properties.cu


In [12]:
!nvcc -o get-device-properties 01-get-device-properties.cu -run

Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32


### 실습: SM 숫자에 맞춘 크기의 그리드를 사용한 벡터합 최적화

앞에서 배운 SM 개수 질의 기능을 이용하여 [`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)의 `addVectorsInto` 커널을 수정하되, 이를 SM 개수의 배수에 해당하는 블럭을 포함한 그리드로 구동되게 하세요.

여러분이 작성하는 코드의 세부 내용에 따라서 수정 사항이 커널 성능 개선을 이룰 수도, 이루지 않을 수도 있습니다. `nvprof`를 이용하여 성능 변화를 정량적으로 측정하고 그 결과를 통해 발견한 사항을 기록하세요.

In [13]:
%%writefile 01-vector-add.cu

#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;
 
  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);
  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = warpSize*32;
  numberOfBlocks = multiProcessorCount*32;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}


Overwriting 01-vector-add.cu


In [14]:
!nvcc -o sm-optimized-vector-add 01-vector-add.cu -run

Success! All values calculated correctly.


In [15]:
!nvprof ./sm-optimized-vector-add

==328== NVPROF is profiling process 328, command: ./sm-optimized-vector-add
Success! All values calculated correctly.
==328== Profiling application: ./sm-optimized-vector-add
==328== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  108.88ms         1  108.88ms  108.88ms  108.88ms  addVectorsInto(float*, float*, float*, int)
      API calls:   70.26%  311.97ms         3  103.99ms  21.221us  311.89ms  cudaMallocManaged
                   24.53%  108.89ms         1  108.89ms  108.89ms  108.89ms  cudaDeviceSynchronize
                    5.04%  22.395ms         3  7.4650ms  6.8266ms  8.6679ms  cudaFree
                    0.08%  353.67us         1  353.67us  353.67us  353.67us  cuDeviceTotalMem
                    0.04%  159.98us       101  1.5830us     132ns  70.387us  cuDeviceGetAttribute
                    0.03%  113.79us         1  113.79us  113.79us  113.79us  cudaGetDeviceProperties
                    0.0

---

## 통합 메모리(Unified Memory) 세부 내용

여러분은 `cudaMallocManaged` 함수를 이용하여 호스트와 디바이스 코드가 사용할 메모리를 할당해 왔습니다. 지금까지 이 함수가 제공하는 자동 메모리 마이그레이션, 쉬운 프로그래밍과 같은 혜택을 이용해 오면서도 `cudaMallocManaged`가 실제로 할당하는 **통합 메모리(unified memeory; UM)**에 대한 세부 내용은 신경 쓸 필요가 없었습니다. `nvprof`는 가속화 애플리케이션의 UM 관리에 대한 상세한 정보를 제공하는데, 이 정보와 아울러 UM의 작동 원리를 보다 잘 이해하게 된다면 가속 애플리케이션 최적화의 추가적인 기회 얻을 수 있습니다.  

다음 슬라이드는 앞으로 공부할 내용을 개략적인 수준에서 시각적으로 보여줍니다. 주제를 보다 상세히 다루기에 앞서 슬라이드를 클릭하시면서 살펴 보세요.

In [16]:
%%HTML

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTasuq4eIe8Xd_G-xL-dD6hbkv48C_8xD4WS1780qnWnidDc5FApS--f86luAU5uM5IlJiiAhBAH4v-/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

### 통합 메모리(UM) 마이그레이션

UM이 할당될 때, 메모리는 아직 호스트 또는 디바이스에 적재되지 않습니다. 호스트나 디바이스가 그 메모리에 접근하려고 하면 [페이지 폴트](https://en.wikipedia.org/wiki/Page_fault)가 일어나고 이 시점에 호스트와 디바이스는 필요한 데이터를 연속적으로 읽어들여 옵니다. 이것을 메모리 마이그레이션(memory migration)이라고 합니다. 이와 마찬가지로 호스트나 디바이스가 아직 적재되지 않은 메모리에 접근을 시도한다면 페이지 폴트가 일어나고 마이그레이션이 시작됩니다.

페이지 폴트와 요청시 마이그레이션은 가속화 애플리케이션 개발을 쉽게 해주어 큰 도움이 됩니다. 특히 애플리케이션이 실제로 실행되어 데이터를 필요로 할 때까지 어느 데이터가 필요한지 알 수 없는 경우가 있는데, 이와 같이 흩어진 데이터를 다루는 경우, 또는 다수의 GPU가 접근하는 데이터와 같은 경우에 있어서 요청시 마이그레이션은 매우 유용합니다.

하지만 어떤 데이터가 필요한지 미리 알 수 있는 경우와 큰 연속 메모리 영역이 필요한 경우도 많이 있습니다. 이 경우에는 오히려 페이지 폴트와 요청시 마이그레이션의 오버헤드가 큰 부담이 되므로 이러한 오버헤드 비용은 피하는 것이 좋습니다.

본 강좌의 나머지 부분은 요청시 마이그레이션을 이해하고 프로파일러 출력에서 이를 확인하는 것에 대해 할애할 것입니다. 이러한 지식을 통하여 여러분은 오버헤드를 피하는 것이 유리한 경우를 이해하고 대처할 수 있게 될 것입니다.



### 실습: UM 페이지 폴트 살펴 보기

`nvprof`는 대상 애플리케이션의 UM 작동 상황을 출력합니다. 본 실습에서 여러분은 간단한 애플리케이션을 수정해 가면서, 각 수정에 대한 `nvprof`의 UM 출력 부분을 이용하여 UM 데이터 마이그레이션 행동을 살펴볼 것입니다.

[`01-page-faults.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/06-unified-memory-page-faults/01-page-faults.cu)은 `hostFunction`과 `gpuKernel`을 포함하는데 이들은 2<<24 개의 원소를 가지는 벡터의 원소값을 1로 초기화하는 역할을 합니다. 아직은 이들 호스트 함수와 디바이스 커널은 사용되지 않습니다. 아래 네 개의 질문에 대하여 여러분이 UM 동작에 대해 배운 것을 기반으로 하여 어떤 종류의 페이지 폴트가 일어날지 가정해 보새요. 다음으로는 [`01-page-faults.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/06-unified-memory-page-faults/01-page-faults.cu)를 수정하되, 앞에서 나온 함수들을 이용하여 여러분의 가정을 검증할 시나리오를 작성하세요.

여러분의 가정을 검증하기 위해서 아래 셀을 이용하여 코드를 컴파일하고 프로파일링 하세요. 여러분의 가정과 함께 `nvprof`에서 얻은 결과를 기록하되, CPU와 GPU의 페이지 폴트에 집중하며 네 개의 실험을 수행하세요. 어떻게 해야 할 지 모를 때에는 질문 옆에 있는 솔루션 링크를 참고하세요. 

* 통합 메모리가 CPU에 의해서만 사용될 때 무슨 일이 일어날까요? ***(솔루션:01-page-faults-solution-cpu-only.cu)***
* 통합 메모리가 GPU에 의해서만 사용될 때 무슨 일이 일어날까요? ***(솔루션:02-page-faults-solution-gpu-only.cu)***
* 통합 메모리가 CPU에 의해 먼저 사용되고 다음으로 GPU에 의해서 사용될 때 무슨 일이 일어날까요? ***(솔루션:03-page-faults-solution-cpu-then-gpu.cu)***
* 통합 메모리가 GPU에 의해 먼저 사용되고 다음으로 CPU에 의해서 사용될 때 무슨 일이 일어날까요? ***(솔루션:04-page-faults-solution-gpu-then-cpu.cu)***

In [17]:
%%writefile 01-page-faults.cu

__global__
void deviceKernel(int *a, int N)
{
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for (int i = idx; i < N; i += stride)
  {
    a[i] = 1;
  }
}

void hostFunction(int *a, int N)
{
  for (int i = 0; i < N; ++i)
  {
    a[i] = 1;
  }
}

int main()
{

  int N = 2<<24;
  size_t size = N * sizeof(int);
  int *a;
  cudaMallocManaged(&a, size);
 
 // hostFunction(a,N);
  hostFunction(a,N);
  deviceKernel<<<128,256>>>(a,N);
  cudaDeviceSynchronize();
  /*
   * Conduct experiments to learn more about the behavior of
   * `cudaMallocManaged`.
   *
   * What happens when unified memory is accessed only by the GPU?
   * What happens when unified memory is accessed only by the CPU?
   * What happens when unified memory is accessed first by the GPU then the CPU?
   * What happens when unified memory is accessed first by the CPU then the GPU?
   *
   * Hypothesize about UM behavior, page faulting specificially, before each
   * experiement, and then verify by running `nvprof`.
   */

  cudaFree(a);
}


Writing 01-page-faults.cu


In [18]:
!nvcc  -o page-faults 01-page-faults.cu -run

In [19]:
!nvprof ./page-faults

==378== NVPROF is profiling process 378, command: ./page-faults
==378== Profiling application: ./page-faults
==378== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  85.614ms         1  85.614ms  85.614ms  85.614ms  deviceKernel(int*, int)
      API calls:   77.72%  324.80ms         1  324.80ms  324.80ms  324.80ms  cudaMallocManaged
                   20.49%  85.643ms         1  85.643ms  85.643ms  85.643ms  cudaDeviceSynchronize
                    1.63%  6.8052ms         1  6.8052ms  6.8052ms  6.8052ms  cudaFree
                    0.09%  392.07us         1  392.07us  392.07us  392.07us  cuDeviceTotalMem
                    0.04%  157.16us       101  1.5560us     140ns  66.800us  cuDeviceGetAttribute
                    0.01%  54.053us         1  54.053us  54.053us  54.053us  cudaLaunchKernel
                    0.01%  29.293us         1  29.293us  29.293us  29.293us  cuDeviceGetName
                    0.0

### 실습: 벡터합 프로그렘에서 UM 동작 다시 살펴보기


이제 다시 [`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)로 돌아오세요. 현재의 코드 베이스를 살펴 보고 어떤 종류의 페이지 폴트가 일어날지 생각해 보세요. 제일 마지막으로 코드를 수정했을 때의 프로파일링 결과를 살펴 보세요. 과거 출력을 찾아 보기 위해 위로 스크롤하시거나 아래 셀에서 명령을 다시 실행하면 됩니다. 통합 메모리의 프로파일 출력을 살펴 보세요. 현재 코드 베이스의 내용으로부터 프로파일의 페이지 폴트 결과 출력을 스스로 설명할 수 있으시겠어요?


In [20]:
!nvprof ./sm-optimized-vector-add

==391== NVPROF is profiling process 391, command: ./sm-optimized-vector-add
Success! All values calculated correctly.
==391== Profiling application: ./sm-optimized-vector-add
==391== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  118.14ms         1  118.14ms  118.14ms  118.14ms  addVectorsInto(float*, float*, float*, int)
      API calls:   68.38%  305.94ms         3  101.98ms  14.897us  305.88ms  cudaMallocManaged
                   26.41%  118.16ms         1  118.16ms  118.16ms  118.16ms  cudaDeviceSynchronize
                    5.03%  22.511ms         3  7.5038ms  6.7355ms  8.7005ms  cudaFree
                    0.09%  394.77us         1  394.77us  394.77us  394.77us  cuDeviceTotalMem
                    0.03%  150.60us       101  1.4910us     142ns  65.391us  cuDeviceGetAttribute
                    0.03%  118.96us         1  118.96us  118.96us  118.96us  cudaGetDeviceProperties
                    0.0

### 실습: 커널에서 벡터 초기화

`nvprof`가 얼마 동안 커널을 실행하여 HtoD(Host to Device) 페이지 폴트와 데이터 마이그레이션이 발생하면, 그 페이지 폴트와 마이그레이션 역시 실행 시간에 포함되어 출력됩니다. 이것을 염두하시면서 [`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)의 `initWith` 호스트 함수를 수정하여 CUDA 커널로 만들고, 할당된 벡터를 GPU에서 병렬로 초기화하세요. 수정된 코드를 성공적으로 컴파일하고 실행한 뒤, 프로파일링에 앞서 다음 사항에 대한 가정을 수립해 보세요.

* 코드 수정이 UM 페이지 폴트 동작에 어떤 영향을 미칠까요?
* 코드 수정이 `addVectorsInto`의 실행 시간에 어떤 영향을 미칠까요?

결과를 기록하세요. 어떻게 해야 할 지 모를 때에는 ***(솔루션:01-vector-add-init-in-kernel-solution.cu)***을 참고하세요.

In [21]:
%%writefile 01-vector-add.cu

#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

__global__ void initWith(float num, float *a, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;
 
  for(int i = index; i < N; i += stride)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;
 
  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);
  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = warpSize*32;
  numberOfBlocks = multiProcessorCount*32;
 
  initWith<<<numberOfBlocks, threadsPerBlock>>>(3, a, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(4, b, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(0, c, N);

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}


Overwriting 01-vector-add.cu


In [22]:
!nvcc -o initialize-in-kernel 01-vector-add.cu -run

Success! All values calculated correctly.


In [23]:
!nvprof ./initialize-in-kernel

==439== NVPROF is profiling process 439, command: ./initialize-in-kernel
Success! All values calculated correctly.
==439== Profiling application: ./initialize-in-kernel
==439== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   97.11%  55.082ms         3  18.361ms  16.580ms  21.861ms  initWith(float, float*, int)
                    2.89%  1.6421ms         1  1.6421ms  1.6421ms  1.6421ms  addVectorsInto(float*, float*, float*, int)
      API calls:   80.19%  320.21ms         3  106.74ms  15.258us  320.13ms  cudaMallocManaged
                   14.21%  56.721ms         1  56.721ms  56.721ms  56.721ms  cudaDeviceSynchronize
                    5.40%  21.542ms         3  7.1808ms  6.3043ms  8.8109ms  cudaFree
                    0.11%  424.89us         1  424.89us  424.89us  424.89us  cuDeviceTotalMem
                    0.04%  157.39us       101  1.5580us     143ns  66.617us  cuDeviceGetAttribute
                    0.03

---
## 비동기 메모리 프리패칭

페이지 폴트와 호스트에서 디바이스로 또는 디바이스에서 호스트로의 메모리 전달인 요구시 메모리 마이그레이션으로 인한 오버헤드를 줄이는 강력한 기법을 **비동기 메모리 프리패칭(asynchronous memory prefetching)**이라고 합니다. 이 기법을 이용하면 애플리케이션이 메모리를 사용하려고 하기 전에 프로그래머가 UM을 임의의 CPU나 GPU로 마이그레이션할 수 있는데, 이는 비동기적으로 백그라운드에서 수행됩니다. 이를 통해 줄어든 페이지 폴트 및 요구시 마이그레이션만큼 GPU 커널과 CPU 함수의 성능이 향상됩니다.

프리패칭은 데이터를 큰 덩어리로 마이그레이션하는 경향이 있어 요구시 마이그레이션보다 메모리 전달 빈도가 낮아집니다. 이는 접근할 데이터를 런타임 전에 알 수 있고, 데이터 접근 패턴이 산발적이지 않은 경우에 아주 적합합니다.

CUDA는 `cudaMemPrefetchAsync` 함수를 이용하여 메모리에서 GPU/CPU로의 비동기적 프리패칭을 손쉽게 처리합니다. 아래 예제는 현재 활성화된 GPU로 데이터를 프리패치한 후, CPU로 프리패치하는 방법을 보여 줍니다.

```cpp
int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.
```



### 실습: 메모리 프리패치하기

지금쯤이면 여러분의 [01-vector-add.cu](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu) 프로그램은 두 개의 벡터를 더하여 세 번째 솔루션 벡터에 대입하는 CUDA 커널을 구동하며, 벡터들은 모두 `cudaMallocManaged`를 이용하여 할당되어 있을 것입니다. 그뿐만 아니라, 3 개의 벡터들은 모두 CUDA 커널에서 병렬로 초기화되어 있을 것입니다. 만약 여러분의 코드가 그렇게 되어 있지 않다면 [참조 코드](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/08-prefetch/01-vector-add-prefetch.cu)를 보시고 여러분의 코드를 수정하세요. 

여러분의 [`01-vector-add.cu`](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/01-vector-add/01-vector-add.cu)에서 `cudaMemPrefetchAsync`를 이용하고, 이 함수가 페이지 폴트와 메모리 마이그레이션에 미치는 영향을 세 가지 실험을 통해 알아 보세요. 

* 초기화된 벡터 중 하나를 호스트로 프리패치하면 어떤 일이 벌어지나요?
* 초기화된 벡터 중 두 개를 호스트로 프리패치하면 어떤 일이 벌어지나요?
* 초기화된 벡터 세 개 모두를 호스트로 프리패치하면 어떤 일이 벌어지나요?

UM의 동작, 특히 페이지 폴트에 대한 가정과 커널 초기화 실행 시간에 미치는 영향에 대한 가정을 수립한 후, `nvprof`를 이용하여 실험을 하고 가정을 검증하세요. 어떻게 해야 할 지 모를 때에는 ***(솔루션:01-vector-add-prefetch-solution.cu)***을 참고하세요.

In [24]:
%%writefile 01-vector-add.cu

#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;
 
  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);

  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);
 
  cudaMemPrefetchAsync(a, size, deviceId);
  cudaMemPrefetchAsync(b, size, deviceId);
  cudaMemPrefetchAsync(c, size, deviceId);
 
  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = warpSize*32;
  numberOfBlocks = multiProcessorCount*32;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}


Overwriting 01-vector-add.cu


In [25]:
!nvcc -o prefetch-to-gpu 01-vector-add.cu -run

Success! All values calculated correctly.


In [26]:
!nvprof ./prefetch-to-gpu

==490== NVPROF is profiling process 490, command: ./prefetch-to-gpu
Success! All values calculated correctly.
==490== Profiling application: ./prefetch-to-gpu
==490== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  1.6351ms         1  1.6351ms  1.6351ms  1.6351ms  addVectorsInto(float*, float*, float*, int)
      API calls:   84.68%  329.23ms         3  109.74ms  13.232us  329.16ms  cudaMallocManaged
                    6.56%  25.491ms         1  25.491ms  25.491ms  25.491ms  cudaDeviceSynchronize
                    5.47%  21.256ms         3  7.0855ms  6.3819ms  8.4144ms  cudaFree
                    3.10%  12.059ms         3  4.0197ms  13.107us  11.905ms  cudaMemPrefetchAsync
                    0.09%  348.15us         1  348.15us  348.15us  348.15us  cuDeviceTotalMem
                    0.04%  154.11us         1  154.11us  154.11us  154.11us  cudaGetDeviceProperties
                    0.04%  153.10us    

### 실습: CPU로 메모리 프리패치하기 

`addVectorIntoAdd` 커널이 잘 동작하는지 검증하는 함수를 위해 CPU로의 프리패칭을 추가하세요. `nvprog`를 이용하여 결과를 확인하기 전에 UM에 미치는 영향에 대한 가정을 수립하세요. 어떻게 해야 할 지 모를 때에는 ***(솔루션:02-vector-add-prefetch-solution-cpu-also.cu)***을 참고하세요.


In [27]:
%%writefile 01-vector-add.cu

#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;
 
  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);

  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);
 
  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nvprof should register performance changes when execution configuration
   * is updated.
   */

  threadsPerBlock = warpSize*32;
  numberOfBlocks = multiProcessorCount*32;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));
  cudaMemPrefetchAsync(c, size, cudaCpuDeviceId);

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}


Overwriting 01-vector-add.cu


In [28]:
!nvcc -o prefetch-to-cpu 01-vector-add.cu -run

Success! All values calculated correctly.


In [29]:
!nvprof ./prefetch-to-cpu

==541== NVPROF is profiling process 541, command: ./prefetch-to-cpu
Success! All values calculated correctly.
==541== Profiling application: ./prefetch-to-cpu
==541== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  109.74ms         1  109.74ms  109.74ms  109.74ms  addVectorsInto(float*, float*, float*, int)
      API calls:   68.32%  315.20ms         3  105.07ms  12.490us  315.14ms  cudaMallocManaged
                   23.79%  109.75ms         1  109.75ms  109.75ms  109.75ms  cudaDeviceSynchronize
                    5.37%  24.789ms         3  8.2629ms  6.6061ms  10.922ms  cudaFree
                    2.35%  10.857ms         1  10.857ms  10.857ms  10.857ms  cudaMemPrefetchAsync
                    0.08%  362.68us         1  362.68us  362.68us  362.68us  cuDeviceTotalMem
                    0.03%  159.09us       101  1.5750us     137ns  67.150us  cuDeviceGetAttribute
                    0.03%  124.74us       

---
## 요약

이제 여러분은 아래와 같은 학습 목표를 달성하셨습니다.

* **NVIDIA 커맨드라인 프로파일러(nvprof)**를 이용하여 가속화 애플리케이션의 성능 프로파일링하기
* 실행 설정 최적화를 위한 **스트리밍 멀티프로세서**에 대한 체계적 이해
* 페이지 폴트와 데이터 마이그레이션에 관련된 **통합 메모리**의 동작 이해
* 페이지 폴트와 데이터 마이그레이션을 감소시켜 성능을 향상시키는 **비동기적 메모리 프리패칭** 사용
* 반복적 개발 사이클을 적용하여 애플리케이션 가속화와 배치를 신속히 진행하기

배운 내용을 숙지하고 애플리케이션의 반복적 가속화/최적화/배치에 대한 여러분의 능력을 향상시키기 위해 강좌의 마지막 실습을 수행하세요. 완료한 후에는 가급적 다음 세션의 *[고급 주제](https://courses.nvidia.com/api/jupyter/render_notebook/?url=https%3A%2F%2Fdeveloper.download.nvidia.com%2Ftraining%2Fcourses%2FC-AC-01-V1%2FAC_STREAMS_NVVP-kr%2FAC_STREAMS_NVVP-kr.ipynb&images_url=#%EA%B3%A0%EA%B8%89-%EC%A3%BC%EC%A0%9C)*의 내용도 적용해 보세요.

---

## 마지막 실습: 가속화 SAXPY 애플리케이션의 반복적 최적화

기초적인 가속화 [SAXPY](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms#Level_1) 애플리케이션이 [여기](../../../../../edit/tasks/task1/task/02_AC_UM_NVPROF-kr/09-saxpy/01-saxpy.cu)에 준비되어 있습니다. 이 코드는 현재 몇 가지 버그가 있습니다. 이것들을 찾아 수정하고 실행한 뒤 `nvprof`을 이용하여 프로파일링하세요.

다 되셨으면 `saxpy` 커널의 실행 시간을 기록하고 **반복적으로** 최적화를 진행하세요. 반복할 때마다 `nvprof`을 이용하여 코드 수정이 커널 성능과 UM 동작에 미치는 영향을 확인하세요.

강좌에서 배운 기법을 적용해 보세요. 강좌에서 배운 기법을 교재에서만 찾기 보다는, 웹을 활용한 [검색을 통한 문제 해결(effortful retrieval)](http://sites.gsu.edu/scholarlyteaching/effortful-retrieval/)을 통해 학습한 내용을 더 잘 이해하실 수 있으실 것입니다.

여러분의 목표는 올바른 saxpy 커널을 프로파일링하되, `N`을 바꾸지 않은 상태에서 *50us* 내에 실행을 완료하는 것입니다. 어떻게 해야 할 지 모를 때에는 ***(솔루션:02-saxpy-solution.cu)***을 참고하시고 자유롭게 컴파일하고 프로파일링 해보셔도 됩니다.

In [30]:
%%writefile 01-saxpy.cu

#include <stdio.h>

#define N 2048 * 2048 // Number of elements in each vector

/*
 * Optimize this already-accelerated codebase. Work iteratively,
 * and use nvprof to support your work.
 *
 * Aim to profile `saxpy` (without modifying `N`) running under
 * 20us.
 *
 * Some bugs have been placed in this codebase for your edification.
 */

__global__ void saxpy(int * a, int * b, int * c)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
 
    for(int i = tid; i<N; i+=stride)
        c[i] = 2 * a[i] + b[i];

}

int main()
{
    int *a, *b, *c;
 
    int deviceId;
    cudaGetDevice(&deviceId);    
    cudaDeviceProp props;
    cudaGetDeviceProperties(&props, deviceId); 
    int multiProcessorCount = props.multiProcessorCount;
    int warpSize = props.warpSize;
 
    int size = N * sizeof (int); // The total number of bytes per vector

    cudaMallocManaged(&a, size);
    cudaMallocManaged(&b, size);
    cudaMallocManaged(&c, size);

    // Initialize memory
    for( int i = 0; i < N; ++i )
    {
        a[i] = 2;
        b[i] = 1;
        c[i] = 0;
    }
    cudaMemPrefetchAsync(a, size, deviceId);
    cudaMemPrefetchAsync(b, size, deviceId);
    cudaMemPrefetchAsync(c, size, deviceId); 

    int threads_per_block = warpSize*32;
    int number_of_blocks = multiProcessorCount*32;

    saxpy <<< number_of_blocks, threads_per_block >>> ( a, b, c );
    cudaDeviceSynchronize();

    // Print out the first and last 5 values of c for a quality check
    for( int i = 0; i < 5; ++i )
        printf("c[%d] = %d, ", i, c[i]);
    printf ("\n");
    for( int i = N-5; i < N; ++i )
        printf("c[%d] = %d, ", i, c[i]);
    printf ("\n");

    cudaFree( a ); cudaFree( b ); cudaFree( c );
}


Writing 01-saxpy.cu


In [31]:
!nvcc -o saxpy 01-saxpy.cu -run

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 


In [32]:
!nvprof ./saxpy

==594== NVPROF is profiling process 594, command: ./saxpy
c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 
==594== Profiling application: ./saxpy
==594== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  195.80us         1  195.80us  195.80us  195.80us  saxpy(int*, int*, int*)
      API calls:   97.37%  326.79ms         3  108.93ms  22.760us  326.73ms  cudaMallocManaged
                    1.27%  4.2729ms         1  4.2729ms  4.2729ms  4.2729ms  cudaDeviceSynchronize
                    0.76%  2.5516ms         3  850.53us  828.60us  881.85us  cudaFree
                    0.37%  1.2370ms         3  412.33us  8.2510us  1.0793ms  cudaMemPrefetchAsync
                    0.12%  393.70us         1  393.70us  393.70us  393.70us  cuDeviceTotalMem
                    0.05%  152.69us       101  1.5110us     133ns  64.456us  cuDeviceGetAt

### HW6

아래에 있는 코드는 파동방정식의 코드입니다.(CPU only)

해당 코드를 위에서 배운 지식들을 활용하여 최대한 가속화하십시오.

그리고 위 코드를 가속화한 방법들에 대한 보고서를 자세히 작성하시오.

***본인의 HW5 코드를 토대로 수정하시면 되고, 프로파일러를 토대로 하여 여러번***

***반복하여 최대한 가속화를 진행해보시기 바랍니다.***

코드와 보고서를 모두 블랙보드상에 10/13(화)까지 제출하시면 됩니다.

파일제목양식 : 2020123456_홍길동_HW06

In [36]:
%%writefile hw6.cu
#include<stdio.h>
#include<math.h>
#include<stdlib.h>

#define PI 3.141592

__global__ void Hyperbolic(float* U, float* U_new, int CFL, int N)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int stride = gridDim.x * blockDim.x;
    for (int i = idx+1; i < N - 1; i+=stride)
        U_new[i] = U[i] - CFL*(U[i] - U[i - 1]);
}

__global__ void Init(float* U,float* U_new, int N, int iter, float dx)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    int stride = gridDim.x * blockDim.x;
    for (int i = idx; i < N - 1; i+=stride)
        U[i] = U_new[i] = 0;
    for (int i = (int)((50) / dx)+idx; i < (int)((110.0 / dx) + 1); i+=stride)
        U[i] = fabs(100.0 * (sin(PI * (i*dx - 50.0) / 60.0)));
}

int main()
{
    float dx = 0.5;
    float dt = 0.02;
    float end_t = 6.0;
    int L = 400;
    int N = (int)((float)L / dx);
    int iter = (int)(end_t / dt);
    float CFL = 1.0;

    float *U, *U_new;

    const int size = N * sizeof(float);

    int deviceId;
    cudaGetDevice(&deviceId);
    cudaDeviceProp props;
    cudaGetDeviceProperties(&props, deviceId);
    int multiProcessorCount = props.multiProcessorCount;
    int warpSize = props.warpSize;

    cudaMallocManaged(&U, size);
    cudaMallocManaged(&U_new, size);
  
    size_t threads_per_block = warpSize*32;
    size_t number_of_blocks = multiProcessorCount*32;

    Init<<<number_of_blocks,threads_per_block>>>(U, U_new, N, iter, dx);
    cudaDeviceSynchronize();

    for (int i = 0; i < N; i++)
        printf("%.3f ", U[i]);
    printf("\n\n\n");

	for (int i = 0; i < iter; i++)
	{
		Hyperbolic<<<number_of_blocks,threads_per_block>>>(U, U_new, CFL, N);
    cudaDeviceSynchronize();
		for (int j = 0; j < N; j++)
			U[j] = U_new[j];
	}


	for (int i = 0; i < N; i++)
		printf("%.3f ", U[i]);
	printf("\n");

	cudaFree(U);
	cudaFree(U_new);
	return 0;
}

Overwriting hw6.cu


In [37]:
!nvcc -o hw6 hw6.cu -run

0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.618 5.234 7.846 10.453 13.053 15.643 18.224 20.791 23.345 25.882 28.402 30.902 33.381 35.837 38.268 40.674 43.051 45.399 47.716 50.000 52.250 54.464 56.641 58.779 60.876 62.932 64.945 66.913 68.835 70.711 72.537 74.314 76.041 77.715 79.335 80.902 82.413 83.867 85.264 86.603 87.882 89.101 90.259 91.355 92.388 93.358 94.264 95.106 95.882 96.593 97.237 97.815 98.325 98.769 99.144 99.452 99.69

In [38]:
!nvprof ./hw6

==697== NVPROF is profiling process 697, command: ./hw6
0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 2.618 5.234 7.846 10.453 13.053 15.643 18.224 20.791 23.345 25.882 28.402 30.902 33.381 35.837 38.268 40.674 43.051 45.399 47.716 50.000 52.250 54.464 56.641 58.779 60.876 62.932 64.945 66.913 68.835 70.711 72.537 74.314 76.041 77.715 79.335 80.902 82.413 83.867 85.264 86.603 87.882 89.101 90.259 91.355 92.388 93.358 94.264 95.106 95.88